# Ввод как изменяемый параметр

Картинки хороши тем, что они не дискретные и имеют хорошие градиенты.

## DeepDream (июль 2015)

Если двигать ввод в сторону улучшения какого-то класса, то изображение начнет изменяться так, чтобы сеть его воспринимало как.

Сеть обучается расознавать разные объекты. Уже обученную можно использовать «наоборот»: изменять входное изображение так, чтобы оно, например, было больше похоже на фотографию котика (предсказанная вероятность нейрона максимизируется). 

Если каждый раз идти по этом градиенту маленьким шагами, то сгенерятся сюрреалистические и психоделические картинки. На них, обычно, очень много объектов нужного класса в самых неожиданных местах.

Обучение происходит чисто через backprop, только вместо фиксированного входа и обучаемых весов происходит наоборот — вход можно оптимизировать, а веса остаются.

In [ ]:
import torch
import torch.nn.functional as F
import argparse

import matplotlib.pyplot as plt

parser = argparse.ArgumentParser()

aprser.add_argument('image', help='Path to modified image.')
aprser.add_argument('model', help='Path to trained inception model.')
parser.add_argument('--target_label', default=0,
                    help='Number of target label. Default is a cat.')

parser.add_argument('--num_iterations', default=50)
parser.add_argument('--learning_rate', default=0.01)

parser.add_argument('--gif_duration', default=2)
parser.add_argument('--gif_delay', default=1)
parser.add_argument('--gif_filename', default='dream.gif')

args = parser.parse_arguments()


device = torch.device(args.device)

img = torch.Tensor(load_img(args.image), trainable=True).to(device)
model = load_model(args.model).to(device)

target = torch.zeros(args.num_labels)
target[args.target_label] = 1

def make_gif(images):
    pass

def get_image(img):
    return img.reshape(3, 140, 140).cpu().numpy()

for _ in range(args.num_iterations):
    probs = model.forward(img)
    loss = F.crossentropy(probs, target)
    loss.backward()

    # Теперь интересная часть. На финальных стадиях лосс и так очень маленький.
    # Значит, градиенты тоже будут маленькими.
    # Отнормируем его так, чтобы делать шаги одинакового размера.
    # Градиент теперь нам подсказывает направление, но не размер шага

    lr_scaled = args.learning_rate / img.grad.data.cpu().numpy().abs().mean()
    #                                ^        Офигенная строчка, да?        ^
    img -= lr_scaled * img.grad.data

    images += [get_image(img)]

make_gif(images, args.gif_filename)

## Neural Style Transfer (январь 2016)

Как известно, фичи в сетях имеют. 

Автору до сих пор не до конца понятно, почему.

Считается матрица Грама — в линейной алгебре это скалярные умножения. Это матрица показывают, как фичи коррелируют между собой.

Важно: сеть уже обучена на чем-то. На имаджнете, например.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.data import Dataset, Dataloader

import matplotlib.pyplot as plt


parser = argparse.ArgumentParser()

aprser.add_argument('folder', help='Path to folder with images.')

parser.add_argument('--num_iterations', default=50)
parser.add_argument('--learning_rate', default=0.01)

parser.add_argument('--gif_duration', default=2)
parser.add_argument('--gif_delay', default=1)
parser.add_argument('--gif_filename', default='dream.gif')

parser.add_argument('--discriminator_iters', default='dream.gif')

args = parser.parse_arguments()


device = torch.device(args.device)

img = torch.Tensor(load_img(args.image), trainable=True).to(device)
model = load_model(args.model).to(device)

target = torch.zeros(args.num_labels)
target[args.target_label] = 1

dataset = ImageFolder(args.folder)

loader = Dataloader(dataset, args.batch_size)

G = nn.Sequential(
    nn.Linear(),
    nn.ReLU(),
    nn.Linear(),
    nn.ReLU(),
    Reshape(),
    nn.Sigmoid()
)

D = nn.Sequential(
    
)


optimizer = torch.optim.Adam(model.params(), lr=args.learning_rate)

for epoch in range(args.num_epochs):
    for i, real in enumerate(loader):
        optimizer.zero_grad()
        noise = torch.randn(args.batch_size, args.noise_dim)
        fake = model.forward(noise)

        real_labels = torch.ones(args.batch_size)
        fake_labels = torch.zeros(args.batch_size)

        X = # concat images
        Y = # concat labels

        loss = F.crossentropy(D(X), Y)
        loss.backward()

        optimizer.step()

        if i % args.sample_every == 0:
            print('%d-%d: %.4f $.4f' % (epoch, i, loss_G.item(), loss_D.item()))
            # save images that are in the batch
    
        # Кормить вместе однородные данные — быстрее

        

make_gif(images, args.gif_filename)

## Adversarial атаки

![panda](images/panda.jpg)

Оказывается, достаточно сдвинуть данные в сторону на совсем чуть-чуть, чтобы перевести её в другой класс по мнению классификатора.

На самом деле, архитектуры нейросетей очень уязвимы к таким атакам.

Можно IRL изготовить какие-нибудь аттрибуты одежды и заставить камеры наблюдения думать, что вы не вы.

Для этого нужно иметь доступ к самой архитектуре.

Достаточно изменить даже один пиксель.

[мем про one hot boi]